In [1]:
import pandas as pd

titanic_train = pd.read_csv('./datasets/train.csv')

In [2]:
titanic_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [5]:
y_train, X_train = titanic_train["Survived"], titanic_train.drop("Survived", axis=1)

In [7]:
titanic_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 66.2+ KB


In [9]:
titanic_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Step1: encoding objects

In [46]:
import re
import math

def cabin_stripper(cabin_data):
    cabins = 
    for cabin in cabin_data:
        if str(cabin) != 'nan':
            cabin = re.sub('[0-9]', "", cabin)
            print(cabin)
        else:
            print("NULL")
    return cabin_data

In [48]:
#cabin_stripper(list(X_train['Cabin']))

In [54]:
#cabin_stripper(X_train['Cabin'][1])

In [58]:
train = pd.DataFrame()
train['PassengerId'] = X_train['PassengerId']

In [59]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
sex_encoded = encoder.fit_transform(titanic_train[['Sex']])
train["Sex"] = sex_encoded

In [62]:
train["Age"] = X_train["Age"]

In [107]:
def get_title(name):
    title_search = re.search(r'([a-zA-Z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""

In [112]:
train['Title'] = X_train['Name'].apply(get_title)

In [118]:
train['Title'].value_counts()

Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Mlle          2
Major         2
Col           2
Ms            1
Don           1
Mme           1
Capt          1
Sir           1
Lady          1
Jonkheer      1
Countess      1
Name: Title, dtype: int64

In [119]:
train['Title'] = train['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

In [124]:
train['Title'].value_counts()

Mr        517
Miss      185
Mrs       126
Master     40
Rare       23
Name: Title, dtype: int64

In [123]:
train['Title'] = train['Title'].replace(['Mlle'], 'Miss')

In [129]:
from sklearn.preprocessing import OneHotEncoder
hot_encoder = OneHotEncoder()
title_encoded = hot_encoder.fit_transform(train[['Title']])

In [132]:
hot_encoder.categories_

[array(['Master', 'Miss', 'Mr', 'Mrs', 'Rare'], dtype=object)]

In [150]:
from sklearn.base import BaseEstimator, TransformerMixin

class TitleExtracter(BaseEstimator, TransformerMixin):
    def __init__(self):
        self = self
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.apply(get_title)
        X = X.replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')        
        X = X.replace('Mlle', 'Miss')
        X = X.replace('Ms', 'Miss')
        X = X.replace('Mme', 'Mrs')
        
        h_encoder = OneHotEncoder()
        encoded_title = h_encoder.fit_transform([X])
        
        return encoded_title
        

In [151]:
t = TitleExtracter()
t.transform(X_train['Name'])

<1x891 sparse matrix of type '<class 'numpy.float64'>'
	with 891 stored elements in Compressed Sparse Row format>

In [158]:
class AttribCombiner(BaseEstimator, TransformerMixin):
    def __init__(self):
        self = self
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        n_x = X[:, 3] + X[:, 4] + 1
        #X.drop(['SibSp', 'Parch'], axis=1)
        return X

In [159]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('combiner', AttribCombiner()),
    ('std_scaler', StandardScaler())
])

In [160]:
from sklearn.compose import ColumnTransformer

n_attribs = ['Pclass','Age','SibSp','Parch','Fare']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, n_attribs),
    ('cat', TitleExtracter(), 'Name'),
    ('car', OrdinalEncoder(), 'Sex')
])

In [161]:
t_prep = full_pipeline.fit_transform(X_train)

ValueError: 1D data passed to a transformer that expects 2D data. Try to specify the column selection as a list of one item instead of a scalar.